# Test the Model and Publish on Hugging Face

## Setup & Imports

In [3]:
from dotenv import load_dotenv
from huggingface_hub import login
from peft import PeftConfig, PeftModel
from twitter_emoji_reaction_lora.model import build_inference_peft_model
import os

In [ ]:
load_dotenv()

login(token=os.getenv("HUGGINGFACE_TOKEN"))

os.environ["WANDB_API_KEY"] = os.getenv("WANDB_API_KEY")
os.environ["WANDB_PROJECT"] = "Emoji-reaction-coach-with-lora"
os.environ["WANDB_NOTES"] = "Fine tune model with low rank adaptation for an emoji reaction coach"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Use only one GPU

In [ ]:
model = build_base_model()
model_id = "roberta-base-with-tweet-eval-emoji" #path to model in local
build_inference_peft_model(model, model_id)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [142]:
id2label = {
    0: "❤", 1: "😍", 2: "😂", 3: "💕", 4: "🔥",
    5: "😊", 6: "😎", 7: "✨", 8: "💙", 9: "😘",
    10: "📷", 11: "🇺🇸", 12: "☀", 13: "💜", 14: "😉",
    15: "💯", 16: "😁", 17: "🎄", 18: "📸", 19: "😜"
}

In [152]:
from transformers import pipeline

pipe = pipeline(
    task="text-classification",
    model=model,
    tokenizer=tok,
    return_all_scores=True,          # return all 20 logits
    function_to_apply="softmax"      # turn logits -> probabilities
)

def emojify(text, k=3):
    probs = pipe(text)[0]
    top = sorted(probs, key=lambda x: x["score"], reverse=True)[:k]
    return " ".join(id2label[int(d["label"].split("_")[-1])] for d in top)


print(emojify("Sunny day"))

Device set to use cuda:0


☀ 💙 😎


In [153]:
repo_id = "codinglabsong/roberta-base-tweet-emoji-lora"

trainer.create_model_card(   # adds automatic metrics, dataset info, tags
    model_name=repo_id.split("/")[-1],
    language="en",
    license="apache-2.0",
    tags=["twitter", "emoji", "lora", "roberta-base"],
)

trainer.push_to_hub(
    repo_id,                        # full path, e.g. "your-hf-username/…"
    finetuned_from="roberta-base",
    tasks="text-classification",
    dataset="tweet_eval/emoji",
)

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/9.51M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/codinglabsong/roberta-base-with-tweet-eval-emoji-full/commit/76696d41cb9ae04b75b3324ac2c921f5b055ae1c', commit_message='codinglabsong/roberta-base-tweet-emoji-lora', commit_description='', oid='76696d41cb9ae04b75b3324ac2c921f5b055ae1c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/codinglabsong/roberta-base-with-tweet-eval-emoji-full', endpoint='https://huggingface.co', repo_type='model', repo_id='codinglabsong/roberta-base-with-tweet-eval-emoji-full'), pr_revision=None, pr_num=None)